# Model Upgrade

## Experimenting with new models

### Initialize imports and datasets

In [ ]:
# External imports
import plotly.graph_objects as go

# from sklearn.metrics import classification_report
import tensorflow as tf
from keras.applications import (
    DenseNet121,
    EfficientNetV2S,
    InceptionResNetV2,
    ResNet50V2,
)
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


# Path to the brain tumor dataset composed of two sub folders training and testing
DATA_DIR = "./data/brain_dataset"
COMBINED_DIR = "./data/combined"

# Path to the brain tumor dataset for testing
TRAINING_DATASET = "./data/brain_dataset/training"
COMBINED_TRAINING_DATASET = "./data/combined/training"

# Path to the brain tumor dataset for testing
TESTING_DATASET = "./data/brain_dataset/testing"
COMBINED_TESTING_DATASET = "./data/combined/testing"

### EfficientNetV2S pre-trained model

#### Model definition and training

In [ ]:
# Charger le modèle pré-entraîné EfficientNetV2S sans la couche de classification
base_model = EfficientNetV2S(
    weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = tf.keras.layers.Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)

# Entraîner le modèle
efficientnet_history = model.fit(
    train_generator, epochs=50, validation_data=validation_generator
)

# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

# Retrieve training and validation accuracy from history
train_acc = efficientnet_history.history["accuracy"]
val_acc = efficientnet_history.history["val_accuracy"]

# Create traces
train_trace = go.Scatter(
    x=list(range(1, len(train_acc) + 1)),
    y=train_acc,
    mode="lines+markers",
    name="Training acc",
    marker=dict(color="darkorange", symbol="circle"),
)

val_trace = go.Scatter(
    x=list(range(1, len(val_acc) + 1)),
    y=val_acc,
    mode="lines+markers",
    name="Validation acc",
    marker=dict(color="green", symbol="square"),
)

# Create layout
layout = go.Layout(
    width=800,
    height=600,
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

# Create figure
fig = go.Figure(data=[train_trace, val_trace], layout=layout)

# Show plot
fig.show()

### InceptionResNetV2 pre-trained model

#### Model definition and training

In [ ]:
# Charger le modèle pré-entraîné InceptionResNetV2 sans la couche de classification
base_model = InceptionResNetV2(
    weights="imagenet", include_top=False, input_shape=(299, 299, 3)
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = tf.keras.layers.Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)

# Entraîner le modèle
inceptionresnet_history = model.fit(
    train_generator, epochs=75, validation_data=validation_generator
)

In [ ]:
# Retrieve training and validation accuracy from history
train_acc = inceptionresnet_history.history["accuracy"]
val_acc = inceptionresnet_history.history["val_accuracy"]

# Create traces
train_trace = go.Scatter(
    x=list(range(1, len(train_acc) + 1)),
    y=train_acc,
    mode="lines+markers",
    name="Training acc",
    marker=dict(color="darkorange", symbol="circle"),
)

val_trace = go.Scatter(
    x=list(range(1, len(val_acc) + 1)),
    y=val_acc,
    mode="lines+markers",
    name="Validation acc",
    marker=dict(color="green", symbol="square"),
)

# Create layout
layout = go.Layout(
    width=800,
    height=600,
    title="InceptionResNetV2 Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

# Create figure
fig = go.Figure(data=[train_trace, val_trace], layout=layout)

# Show plot
fig.show()

# Evaluating the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

In [ ]:
# Charger le modèle pré-entraîné InceptionResNetV2 sans la couche de classification
base_model = InceptionResNetV2(
    weights="imagenet", include_top=False, input_shape=(299, 299, 3)
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = tf.keras.layers.Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATASET, target_size=(299, 299), batch_size=16, class_mode="categorical"
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(299, 299), batch_size=16, class_mode="categorical"
)

# Entraîner le modèle
inceptionresnet_history_v2 = model.fit(
    train_generator, epochs=75, validation_data=validation_generator
)

In [ ]:
# Retrieve training and validation accuracy from history
train_acc = inceptionresnet_history_v2.history["accuracy"]
val_acc = inceptionresnet_history_v2.history["val_accuracy"]

# Create traces
train_trace = go.Scatter(
    x=list(range(1, len(train_acc) + 1)),
    y=train_acc,
    mode="lines+markers",
    name="Training acc",
    marker=dict(color="darkorange", symbol="circle"),
)

val_trace = go.Scatter(
    x=list(range(1, len(val_acc) + 1)),
    y=val_acc,
    mode="lines+markers",
    name="Validation acc",
    marker=dict(color="green", symbol="square"),
)

# Create layout
layout = go.Layout(
    width=800,
    height=600,
    title="InceptionResNetV2 Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

# Create figure
fig = go.Figure(data=[train_trace, val_trace], layout=layout)

# Show plot
fig.show()

# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

In [ ]:
# Charger le modèle pré-entraîné DenseNet121 sans la couche de classification
base_model = DenseNet121(
    weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATASET, target_size=(224, 224), batch_size=16, class_mode="categorical"
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=16, class_mode="categorical"
)

# Entraîner le modèle
history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

In [ ]:
# Use ResNet50V2 from keras.application as pre trained model with imagenet weight
base_model = ResNet50V2(
    weights="imagenet", classes=4, input_shape=(224, 224, 3), include_top=False
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    COMBINED_TRAINING_DATASET,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    COMBINED_TESTING_DATASET,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
)

# Checkpoint filepath to save the model
checkpoint_filepath = "checkpoint.ResNet50V2.keras"

# Checkpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

# Entraîner le modèle avec le callback
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[model_checkpoint_callback],
)

# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    COMBINED_TESTING_DATASET,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

In [ ]:
# Use ResNet50V2 from keras.application as pre trained model with imagenet weight
base_model = ResNet50V2(
    weights="imagenet", classes=4, input_shape=(224, 224, 3), include_top=False
)

# Ajouter une couche GlobalAveragePooling2D pour réduire la dimensionalité
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Ajouter une couche dense pour la classification
predictions = Dense(4, activation="softmax")(x)

# Créer le nouveau modèle en combinant le modèle pré-entraîné et les nouvelles couches
model = Model(inputs=base_model.input, outputs=predictions)

# Congeler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Charger les données d'entraînement et de validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    COMBINED_TRAINING_DATASET,
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical",
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    COMBINED_TESTING_DATASET,
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical",
)

# Checkpoint filepath to save the model
checkpoint_filepath = "checkpoint.ResNet50V2-batch16.keras"

# Checkpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)

# Entraîner le modèle avec le callback
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[model_checkpoint_callback],
)

# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    COMBINED_TESTING_DATASET,
    target_size=(224, 224),
    batch_size=16,
    class_mode="categorical",
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

#### Model evaluation

In [ ]:
# Evaluate the model on the testing dataset
test_loss, test_acc = model.evaluate(validation_generator)
print("Test accuracy:", test_acc)

# Alternatively, if you want to evaluate on a separate test dataset:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TESTING_DATASET, target_size=(224, 224), batch_size=32, class_mode="categorical"
)
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

# Retrieve training and validation accuracy from history
train_acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

# Create traces
train_trace = go.Scatter(
    x=list(range(1, len(train_acc) + 1)),
    y=train_acc,
    mode="lines+markers",
    name="Training acc",
    marker=dict(color="darkorange", symbol="circle"),
)

val_trace = go.Scatter(
    x=list(range(1, len(val_acc) + 1)),
    y=val_acc,
    mode="lines+markers",
    name="Validation acc",
    marker=dict(color="green", symbol="square"),
)

# Create layout
layout = go.Layout(
    width=800,
    height=600,
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

# Create figure
fig = go.Figure(data=[train_trace, val_trace], layout=layout)

# Show plot
fig.show()

In [ ]:
from keras.models import load_model
import numpy as np
from keras import utils

classifier = load_model("ResNet50V2-32.h5")


def new_prediction(path):
    # Charger l'image
    test_image = utils.load_img(path, target_size=(224, 224))

    # Convertir l'image en tableau et redimensionner les pixels
    test_image = utils.img_to_array(test_image)
    test_image /= 255.0

    # Ajouter une dimension de lot à l'image
    test_image = np.expand_dims(test_image, axis=0)

    # Effectuer la prédiction en utilisant la fonction predict du modèle
    result = classifier.predict(test_image)

    # Déterminer la classe prédite et la probabilité maximale
    class_pred = ""
    if np.argmax(result) == 0:
        class_pred = "glioma_tumor"
    elif np.argmax(result) == 1:
        class_pred = "meningioma_tumor"
    elif np.argmax(result) == 2:
        class_pred = "no_tumor"
    elif np.argmax(result) == 3:
        class_pred = "pituitary_tumor"
    return [class_pred, np.max(result)]

In [ ]:
new_prediction("./dataset/combined/testing/glioma_tumor/G_0057.jpg")